# Simple Gen AI APP Using LangChain

In [1]:
# https://docs.langchain.com/langsmith/prompt-engineering-quickstart
import os
from dotenv import load_dotenv
load_dotenv()

open_key = os.getenv("OPENAI_API_KEY")
open_endpoint = os.getenv("OPENAI_API_END")
# langsmith tracking
os.environ['LANGCHAIN_API_KEY'] = os.getenv("LANGCHAIN_API_KEY")
os.environ['LANGCHAIN_TRACING_V2'] = "true"
os.environ['LANGCHAIN_PROJECT'] = os.getenv("LANGCHAIN_PROJECT")

In [3]:
# Data Ingestion - Web Scrapping
from langchain_community.document_loaders import WebBaseLoader

In [4]:
loader = WebBaseLoader(['https://docs.langchain.com/langsmith/prompt-engineering-quickstart'])
print(loader)

In [5]:
docs = loader.load()
print(docs)

[Document(metadata={'source': 'https://docs.langchain.com/langsmith/prompt-engineering-quickstart', 'title': 'Prompt engineering quickstart - Docs by LangChain', 'language': 'en'}, page_content='Prompt engineering quickstart - Docs by LangChainOur new LangChain Academy Course Deep Research with LangGraph is now live! Enroll for free.Docs by LangChain home pagePythonSearch...⌘KLangSmithPlatform for LLM observability and evaluationOverviewQuickstartsTrace an applicationEvaluate an applicationTest promptsAPI & SDKsAPI referencePython SDKJS/TS SDKPricingPlansPricing FAQOur new LangChain Academy Course Deep Research with LangGraph is now live! Enroll for free.Docs by LangChain home pagePythonSearch...⌘KAsk AIForumForumSearch...NavigationQuickstartsPrompt engineering quickstartGet startedObservabilityEvaluationPrompt engineeringSelf-hostingAdministrationGet startedObservabilityEvaluationPrompt engineeringSelf-hostingAdministrationForumOn this pageSDK1. Setup2. Create a prompt3. Test a prompt

In [6]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
documents = text_splitter.split_documents(docs)
documents

[Document(metadata={'source': 'https://docs.langchain.com/langsmith/prompt-engineering-quickstart', 'title': 'Prompt engineering quickstart - Docs by LangChain', 'language': 'en'}, page_content='Prompt engineering quickstart - Docs by LangChainOur new LangChain Academy Course Deep Research with LangGraph is now live! Enroll for free.Docs by LangChain home pagePythonSearch...⌘KLangSmithPlatform for LLM observability and evaluationOverviewQuickstartsTrace an applicationEvaluate an applicationTest promptsAPI & SDKsAPI referencePython SDKJS/TS SDKPricingPlansPricing FAQOur new LangChain Academy Course Deep Research with LangGraph is now live! Enroll for free.Docs by LangChain home pagePythonSearch...⌘KAsk AIForumForumSearch...NavigationQuickstartsPrompt engineering quickstartGet startedObservabilityEvaluationPrompt engineeringSelf-hostingAdministrationGet startedObservabilityEvaluationPrompt engineeringSelf-hostingAdministrationForumOn this pageSDK1. Setup2. Create a prompt3. Test a prompt

In [7]:
from langchain_openai import AzureOpenAIEmbeddings

embeddings = AzureOpenAIEmbeddings(
    model="text-embedding-ada-002",
    azure_endpoint=open_endpoint,
    api_key=open_key,
    api_version="2024-12-01-preview"
)

embeddings

AzureOpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x000001F2FBF10500>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x000001F2FC3F8200>, model='text-embedding-ada-002', dimensions=None, deployment=None, openai_api_version='2024-12-01-preview', openai_api_base=None, openai_api_type='azure', openai_proxy=None, embedding_ctx_length=8191, openai_api_key=SecretStr('**********'), openai_organization=None, allowed_special=None, disallowed_special=None, chunk_size=2048, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None, http_async_client=None, check_embedding_ctx_length=True, azure_endpoint='https://cortizh2-0745-resource.services.ai.azure.com/', azure_ad_token=None, azure_ad_token_provider=None, azure_ad_async_token_provider=None, va

In [8]:
from langchain_community.vectorstores import FAISS

vectorstoredb = FAISS.from_documents(documents, embeddings)

In [9]:
vectorstoredb

In [11]:
query = "The first thing you might want to trace is all your OpenAI calls"
results = vectorstoredb.similarity_search(query)
print(results[0].page_content)

This guide will walk through how to create, test, and iterate on prompts using the SDK and in the UI. In this guide we will use OpenAI, but you can also use other LLM providers.
​SDK
​1. Setup
First, install the required packages:
PythonTypeScriptCopyAsk AIpip install -qU langsmith openai langchain_core


In [12]:
from langchain_openai import AzureChatOpenAI

llm = AzureChatOpenAI(
    model="gpt-5-nano",
    api_version="2024-12-01-preview",
    api_key=open_key,
    azure_endpoint=open_endpoint
)

In [17]:
# Retrieval Chain > Document Chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(
    """
    Answer the following question {input}, based only on the provided context:
    <context>
    {context}
    </context>
"""
)


document_chain = create_stuff_documents_chain(llm, prompt)
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, template='\n    Answer the following question {input}, based only on the provided context:\n    <context>\n    {context}\n    </context>\n'), additional_kwargs={})])
| AzureChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x000001F2C7505A30>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000001F2C73ACFB0>, root_client=<openai.lib.azure.AzureOpenAI object at 0x000001F2C737CC20>, root_async_client=<openai.lib.azure.AsyncAzureOpenAI object at 0x000001F2C7504B60>, model_name='gpt-5-nano', model_kwargs={},

In [19]:
from langchain_core.documents import Document

document_chain.invoke({
    "input": "The first thing you might want to trace is all your OpenAI calls",
    "context": [Document(page_content="""The first thing you might want to trace is all your OpenAI calls this guide will walk through how to create, test, and iterate on prompts using the SDK and in the UI. In this guide we will use OpenAI, but you can also use other LLM providers.
​SDK
​1. Setup
First, install the required packages:
PythonTypeScriptCopyAsk AIpip install -qU langsmith openai langchain_core""")]
})

'All of your OpenAI calls.'

In [20]:
from langchain.chains import create_retrieval_chain

## Input > Retriever < Vector Store DB
retriever = vectorstoredb.as_retriever()

retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [21]:
retrieval_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'AzureOpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001F2FFD182C0>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, template='\n    Answer the following question {input}, based only on the provided context:\n    <context>\n    {context}\n    </context>

In [22]:
# Get Response from the llm

response = retrieval_chain.invoke({"input":"The first thing you might want to trace is all your OpenAI calls"})
print(response)

{'input': 'The first thing you might want to trace is all your OpenAI calls', 'context': [Document(id='63c67ad6-0b38-4524-84df-8703a250c3c7', metadata={'source': 'https://docs.langchain.com/langsmith/prompt-engineering-quickstart', 'title': 'Prompt engineering quickstart - Docs by LangChain', 'language': 'en'}, page_content='This guide will walk through how to create, test, and iterate on prompts using the SDK and in the UI. In this guide we will use OpenAI, but you can also use other LLM providers.\n\u200bSDK\n\u200b1. Setup\nFirst, install the required packages:\nPythonTypeScriptCopyAsk AIpip install -qU langsmith openai langchain_core'), Document(id='8f42901a-6b4f-42fc-94bd-b4c44e2166e4', metadata={'source': 'https://docs.langchain.com/langsmith/prompt-engineering-quickstart', 'title': 'Prompt engineering quickstart - Docs by LangChain', 'language': 'en'}, page_content='Prompt engineering quickstart - Docs by LangChainOur new LangChain Academy Course Deep Research with LangGraph is 

In [26]:
print(response['answer'])

Based on the provided context, the first thing to trace is every OpenAI call. Here's how to approach it using what's in the context:

- Install and set up for tracing
  - Install LangSmith and OpenAI (as shown in the SDK setup).
  - Use the SDK/UI workflow described in the prompt engineering quickstart to connect your app to OpenAI.

- Instrument every OpenAI interaction
  - Ensure each prompt sent to OpenAI and the corresponding response from OpenAI are captured as trace events.
  - Do this for all OpenAI calls your app makes via the SDK/UI.

- Use LangSmith observability and evaluation
  - Leverage LangSmith to trace, evaluate, and iterate on prompts during development and testing.

- Follow the prompt engineering workflow
  - Create a prompt, test a prompt, and iterate on a prompt as outlined in the quickstart.
  - In the UI, you can save and re-test prompts as you refine them.

- Next steps after tracing
  - Learn how to store and manage prompts with the Prompt Hub for ongoing prom